In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
df_fake["class"] = 0
df_true["class"] = 1

df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [8]:
data_merge = pd.concat([df_fake, df_true], axis=0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [11]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [13]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [14]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [16]:
data = data.sample(frac=1)


In [18]:
data.head()

,text,class
7398,"Looking back at all of the offensive, ridiculo...",0
3997,Chicago doesn t tend to be one of the cities o...,0
18176,WASHINGTON (Reuters) - President Donald Trump ...,1
8664,MEXICO CITY (Reuters) - Mexican President Enri...,1
8384,"PALO ALTO, California (Reuters) - Democrat Hil...",1


In [19]:
data.reset_index(inplace=True)
data.drop(['index'], axis = 1, inplace=True)

In [20]:
data.columns

Index(['text', 'class'], dtype='object')

In [21]:
data.head()

,text,class
0,"Looking back at all of the offensive, ridiculo...",0
1,Chicago doesn t tend to be one of the cities o...,0
2,WASHINGTON (Reuters) - President Donald Trump ...,1
3,MEXICO CITY (Reuters) - Mexican President Enri...,1
4,"PALO ALTO, California (Reuters) - Democrat Hil...",1


In [25]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # ลบข้อความที่อยู่ ในวงเล็บเหลี่ยม
    text = re.sub("\\W", " ", text) # แทน อักขระที่ไม่ใช่ตัวอักษรหรือตัวเลข (non-word characters) ด้วยช่องว่าง
    text = re.sub('https?://\S+|www\.\S+','', text) # ลบ ลิงก์ URL 
    text = re.sub('<.*?>+', '', text) # ลบ แท็ก HTML
    text = re.sub('[%s]' % re.escape(string.punctuation),'', text) # ลบ เครื่องหมายวรรคตอนทั้งหมด (เช่น . , ! ?) 
    text = re.sub('\n', '', text) # ลบ ขึ้นบรรทัดใหม่ (new line) ทั้งหมด
    text = re.sub('\w*\d\w*', '', text) # ลบคำที่มี ตัวเลขปะปนอยู่ เช่น word123 (ใช้บ่อยในการลบ token ที่ไม่เป็นคำ เช่น รหัสสินค้า)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\nattaponga\AppData\Local\Temp\ipykernel_18100\2631592762.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text) # ลบข้อความที่อยู่ ในวงเล็บเหลี่ยม
C:\Users\nattaponga\AppData\Local\Temp\ipykernel_18100\2631592762.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+','', text) # ลบ ลิงก์ URL
C:\Users\nattaponga\AppData\Local\Temp\ipykernel_18100\2631592762.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text) # ลบคำที่มี ตัวเลขปะปนอยู่ เช่น word123 (ใช้บ่อยในการลบ token ที่ไม่เป็นคำ เช่น รหัสสินค้า)


In [26]:
text = "Visit my blog at https://myblog.com! This is version v2.0 [note]"
cleaned = wordopt(text)
print(cleaned)


visit my blog at https   myblog com  this is version   


In [28]:
data['text'] = data['text'].apply(wordopt)

In [30]:
x = data['text']
y = data['class']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorrization = TfidfVectorizer()
xv_train = vectorrization.fit_transform(x_train)
xv_test = vectorrization.transform(x_test)

In [37]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [38]:
pred_lr = LR.predict(xv_test)

In [39]:
LR.score(xv_test, y_test)

0.9850334075723831

In [40]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5780
           1       0.98      0.99      0.98      5445

    accuracy                           0.99     11225
   macro avg       0.98      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [41]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [43]:
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)

0.9962583518930958

In [44]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5780
           1       1.00      0.99      1.00      5445

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



In [46]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state= 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [48]:
pred_gb = GB.predict(xv_test)
GB.score(xv_test, y_test)

0.9951893095768374

In [49]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5780
           1       0.99      1.00      1.00      5445

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



In [50]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state=0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [53]:
pred_rf = RF.predict(xv_test)
RF.score(xv_test, y_test)

0.9888641425389755

In [54]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5780
           1       0.99      0.99      0.99      5445

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [55]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not a Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorrization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GB.predict(new_xv_test)
    pred_RFC = RF.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [67]:
news = str(input())
print(news)
manual_testing(news)

Scotland’s Instagram Highway became an overtourism nightmare. It could be worse


LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News
